In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import numpy as np
import os

from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
batch_size =32
epochs = 100
lr = 0.001
train_ratio= 0.9

from sklearn.datasets import load_digits
digits = load_digits()
x_, y_ = digits.data, digits.target

x_ = x_ / x_.max() 

y_one_hot = np.zeros((len(y_), 10))  
y_one_hot[np.arange(len(y_)), y_] = 1
x_.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_, 
                                                    y_one_hot, 
                                                    test_size = 0.05, 
                                                    stratify  = y_)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, 
                                                      y_train, 
                                                      test_size = 1.0 - train_ratio,
                                                      stratify = y_train.argmax(axis = 1))

## Build low level with tensor elements

In [ ]:
import tensorflow as tf

tf.reset_default_graph()

with tf.name_scope('input'):
    x = tf.placeholder(shape = (None, x_train.shape[1]),
                       name = 'x',
                       dtype = tf.float32)
    y = tf.placeholder(shape = (None, y_train.shape[1]), 
                           name = 'y',
                           dtype=tf.float32)

with tf.variable_scope('hidden_layer1'):
    w1 = tf.get_variable('weight1', 
                         shape= [x_train.shape[1], 25], 
                         dtype=tf.float32, 
                         initializer=tf.truncated_normal_initializer(stddev=0.1))
    b1 = tf.get_variable('bias1', 
                         shape= [25], 
                         dtype=tf.float32, 
                         initializer=tf.constant_initializer(0.0))  
    x_h1 = tf.nn.relu(tf.add(tf.matmul(x, w1), b1))

with tf.variable_scope('hidden_layer2'):
    w2 = tf.get_variable('weight2', 
                         shape = [25, 25], 
                         dtype = tf.float32, 
                         initializer = tf.truncated_normal_initializer(stddev=0.1))
    b2 = tf.get_variable('bias2', 
                         shape = [25], 
                         dtype = tf.float32, 
                         initializer = tf.constant_initializer(0.0))  
    x_h2 = tf.nn.relu(tf.add(tf.matmul(x_h1, w2), b2))
    
with tf.variable_scope('hidden_layer3'):
    w3 = tf.get_variable('weight3', 
                         shape = [25, 25], 
                         dtype =tf.float32, 
                         initializer = tf.truncated_normal_initializer(stddev=0.1))
    b3 = tf.get_variable('bias3', 
                         shape = [25], 
                         dtype = tf.float32, 
                         initializer = tf.constant_initializer(0.0))  
    x_h3 = tf.nn.relu(tf.add(tf.matmul(x_h2, w3), b3))

with tf.variable_scope('output_layer'):
    w4 = tf.get_variable('weight4',
                         shape = [25, y_train.shape[1]],
                         dtype = tf.float32,
                         initializer = tf.truncated_normal_initializer(stddev=0.1))
    b4 = tf.get_variable('bias4', 
                         shape = [y_train.shape[1]], 
                         dtype = tf.float32, 
                         initializer = tf.constant_initializer(0.0))
    output = tf.add(tf.matmul(x_h3, w4), b4)
    
with tf.name_scope('cross_entropy'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = output, labels = y))

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(tf.nn.softmax(output), 1), tf.argmax(y, 1)) #如果答案對則回傳truth
    compute_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #將回傳的truth/false轉乘1/0並計算平均(計算正確率)
#-----------------------optimizer---------------------------------
with tf.name_scope('train'):
    #使用adam做optimization最小化loss funciotn(不斷取微分並逼近local min)
    train_step = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

In [ ]:
train_loss_list = []
valid_loss_list = []
train_acc_list = []
valid_acc_list = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in tqdm(range(epochs)):
        iterations = int(np.floor(len(x_train) / batch_size))
        train_loss_collector = []
        train_acc_collector = []
        for j in np.arange(iterations):
            batch_idx_start = j * batch_size
            batch_idx_stop = (j+1) * batch_size
            
            x_batch = x_train[batch_idx_start : batch_idx_stop] 
            y_batch = y_train[batch_idx_start : batch_idx_stop]
            
            this_loss, this_acc, _ = sess.run([loss, compute_acc, train_step], 
                                    feed_dict = {x : x_batch,
                                                 y : y_batch})
            
            train_loss_collector.append(this_loss) #記錄每個batch的loss
            train_acc_collector.append(this_acc)
            
        valid_acc, valid_loss = sess.run([compute_acc, loss],
                                         feed_dict = {x : x_valid,
                                                      y : y_valid})
        valid_loss_list.append(valid_loss) #記錄validation loss
        valid_acc_list.append(valid_acc)   #記錄validation acc
        train_loss_list.append(np.mean(train_loss_collector)) #記錄每個epoch 平均 loss
        train_acc_list.append(np.mean(train_acc_collector))   #記錄每個epoch 平均 acc

        # at the end of each epoch, shuffle the data 重新排列資料並進入下一個i(epochs)
        x_train, y_train = shuffle(x_train, y_train)
        
    test_acc, test_loss = sess.run([compute_acc, loss],
                                    feed_dict = {x : x_test,
                                                 y : y_test})
print('--- training done ---')
print('testing accuracy: %.2f' % test_acc)
#------------------------------------------------------plot----------------------------------------------------------
plt.plot(np.arange(len(train_loss_list)), train_loss_list, 'b', label = 'train')
plt.plot(np.arange(len(valid_loss_list)), valid_loss_list, 'r', label = 'valid')
plt.legend()
plt.show()

plt.plot(np.arange(len(train_acc_list)), train_acc_list, 'b', label = 'train')
plt.plot(np.arange(len(valid_acc_list)), valid_acc_list, 'r', label = 'valid')
plt.legend(loc = 4)
plt.show()

## Build with "Layer"

In [ ]:
tf.reset_default_graph() # clean graph

with tf.name_scope('input'):
    x = tf.placeholder(shape = (None,x_train.shape[1]), 
                             name = 'x',
                             dtype=tf.float32)
    y = tf.placeholder(shape = (None, y_train.shape[1]), 
                           name = 'y',
                           dtype=tf.float32)

with tf.variable_scope('hidden_layer1'):
    x_h1 = tf.layers.dense(inputs= x, units= 64, activation=tf.nn.relu)

with tf.variable_scope('hidden_layer2'):
    x_h2 = tf.layers.dense(inputs= x_h1, units= 64, activation=tf.nn.relu)
    
with tf.variable_scope('hidden_layer3'):
    x_h3 = tf.layers.dense(inputs= x_h2, units= 64, activation=tf.nn.relu)

with tf.variable_scope('output_layer'):
    output = tf.layers.dense(x_h3, 10)

with tf.name_scope('cross_entropy'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = output, labels = y))
    
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(tf.nn.softmax(output), 1), tf.argmax(y, 1))
    compute_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

train_loss_list = []
valid_loss_list = []
train_acc_list = []
valid_acc_list = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in tqdm(range(epochs)):
        iterations = int(np.floor(len(x_train) / batch_size))
        train_loss_collector = []
        train_acc_collector = []
        for j in np.arange(iterations):
            batch_idx_start = j * batch_size
            batch_idx_stop = (j+1) * batch_size
            
            x_batch = x_train[batch_idx_start : batch_idx_stop] 
            y_batch = y_train[batch_idx_start : batch_idx_stop]
            
            this_loss, this_acc, _ = sess.run([loss, compute_acc, train_step], 
                                    feed_dict = {x : x_batch,
                                                 y : y_batch})
            
            train_loss_collector.append(this_loss) #記錄每個batch的loss
            train_acc_collector.append(this_acc)
            
        valid_acc, valid_loss = sess.run([compute_acc, loss],
                                         feed_dict = {x : x_valid,
                                                      y : y_valid})
        valid_loss_list.append(valid_loss) #記錄validation loss
        valid_acc_list.append(valid_acc)   #記錄validation acc
        train_loss_list.append(np.mean(train_loss_collector)) #記錄每個epoch 平均 loss
        train_acc_list.append(np.mean(train_acc_collector))   #記錄每個epoch 平均 acc

        # at the end of each epoch, shuffle the data 重新排列資料並進入下一個i(epochs)
        x_train, y_train = shuffle(x_train, y_train)
        
    test_acc, test_loss = sess.run([compute_acc, loss],
                                    feed_dict = {x : x_test,
                                                 y : y_test})
print('--- training done ---')
print('testing accuracy: %.2f' % test_acc)
#------------------------------------------------------plot----------------------------------------------------------
plt.plot(np.arange(len(train_loss_list)), train_loss_list, 'b', label = 'train')
plt.plot(np.arange(len(valid_loss_list)), valid_loss_list, 'r', label = 'valid')
plt.legend()
plt.show()

plt.plot(np.arange(len(train_acc_list)), train_acc_list, 'b', label = 'train')
plt.plot(np.arange(len(valid_acc_list)), valid_acc_list, 'r', label = 'valid')
plt.legend(loc = 4)
plt.show()